### Predicción de precio de viajes en __Uber__

In [23]:
#importar uber.csv 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import folium

#Importar el archivo
df = pd.read_csv('uber.csv')



In [24]:
df.head(3)

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1


In [25]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.drop(['key'] , axis=1 , inplace=True)
df.head(3)


,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1


Separar la columna de fecha y hora en dos columnas diferentes

In [26]:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df['pickup_date'] = df['pickup_datetime'].dt.date
df['pickup_time'] = df['pickup_datetime'].dt.time
df.drop(['pickup_datetime'] , axis=1 , inplace=True)
df.head(3)

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_date,pickup_time
0,7.5,-73.999817,40.738354,-73.999512,40.723217,1,2015-05-07,19:52:06
1,7.7,-73.994355,40.728225,-73.994710,40.750325,1,2009-07-17,20:04:56
2,12.9,-74.005043,40.740770,-73.962565,40.772647,1,2009-08-24,21:45:00


Categoria de la hora del dia por mañana __(0)__ , tarde __(1)__ , noche __(2)__ y madrugada __(3)__.

In [27]:
df['pickup_time'] = df['pickup_time'].apply(lambda x: x.strftime('%H:%M:%S'))
df['pickup_time'] = pd.to_datetime(df['pickup_time'])


In [28]:
df['pickup_time'] = df['pickup_time'].dt.hour
df['pickup_time'] = df['pickup_time'].astype(int)
df['pickup_time'] = df['pickup_time'].astype(str)
df['pickup_time'] = df['pickup_time'].replace(['0','1','2','3','4','5','6'],'3')
df['pickup_time'] = df['pickup_time'].replace(['7', '8', '9', '10','11','12'],'0')
df['pickup_time'] = df['pickup_time'].replace(['13','14','15','16','17','18'],'1')
df['pickup_time'] = df['pickup_time'].replace(['19','20','21','22','23'],'2')

df.head(3)

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_date,pickup_time
0,7.5,-73.999817,40.738354,-73.999512,40.723217,1,2015-05-07,2
1,7.7,-73.994355,40.728225,-73.994710,40.750325,1,2009-07-17,2
2,12.9,-74.005043,40.740770,-73.962565,40.772647,1,2009-08-24,2


In [31]:
from folium.plugins import MarkerCluster

df2 = df.sample(1000)

# Crear un mapa centrado en la primera ubicación
mymap = folium.Map(location=[df2['pickup_latitude'].iloc[0], df2['pickup_longitude'].iloc[0]], zoom_start=12)

# Agregar un cluster de marcadores para todas las ubicaciones
marker_cluster = MarkerCluster().add_to(mymap)

# Iterar sobre los datos y agregar marcadores
for i in range(len(df2)):
    folium.Marker(location=[df2['pickup_latitude'].iloc[i], df2['pickup_longitude'].iloc[i]],
                  popup=f"Fare Amount: ${df['fare_amount'].iloc[i]:.2f}",
                  icon=folium.Icon(color='blue')).add_to(marker_cluster)

# Calcular la distancia total
df['distance'] = ((df2['pickup_longitude'] - df2['dropoff_longitude'])**2 +
                  (df2['pickup_latitude'] - df2['dropoff_latitude'])**2)**0.5

# Agregar una línea para representar la distancia total
folium.PolyLine(locations=list(zip(df2['pickup_latitude'], df2['pickup_longitude'])),
                color='red', weight=2.5, opacity=1).add_to(mymap)

# Mostrar la distancia total en el popup
total_distance = df2['distance'].sum()
total_fare = df2['fare_amount'].sum()

mymap.add_child(folium.Popup(f"Distancia Total: {total_distance:.2f} unidades\n"
                             f"Total Fare Amount: ${total_fare:.2f}",
                             parse_html=True))

# Guardar el mapa como un archivo HTML
mymap.save('mapa_distancia_total.html')